#Análise de dados do Google Play e Apple Store

##Autores

Alice Soares Pereira de Barros;

Vinicius Santos Silva de Lima.

##Introdução

###Sobre

Esse projeto é sobre um programa que faz uma análise de dados sobre aplicativos da Google Play e Apple Store. 

O programa é feito todo em python 3 usando a ferramenta Colaboratory. 

###Objetivo

O objetivo é limpar os dados recebidos da Google play e da Apple store e identificar qual o tipo de aplicativo gratuito e em inglês é mais comum. 

##Explorando dados

Primeiramente abrimos os arquivos que tem os dados:

In [0]:
from csv import reader

opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
apps_data_apple = list(read_file)

opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
apps_data_google = list(read_file)

A função abaixo imprime informações dos arquivos e de linhas e colunas.

In [0]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

Exemplo com os dados da Google:

In [0]:
explore_data(apps_data_google, 1, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13


Exemplo com os dados da Apple:

In [0]:
explore_data(apps_data_apple, 1, 3, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 7198
Number of columns: 16


##Removendo duplicações da Google play

Agora perceba que alguns apps da Google play estão duplicados: 

In [0]:
for app in apps_data_google:
    name = app[0]
    if name == "Instagram":
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Para remover os duplicados primeiro separamos aqueles que estão duplicados dos que não estão:

In [0]:
duplicate_apps = []
unique_apps = []

for app in apps_data_google[1:]:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
for app in duplicate_apps:
    if app in unique_apps:
        unique_apps.remove(app)

Agora removemos aqueles duplicados que tem menor quantidade avalições, pois queremos trabalhar com os dados mais recentes.

In [0]:
apps_non_duplicates = []

for name in duplicate_apps:
    rating = 0
    for app in apps_data_google:
        if name == app[0]:
            if int(app[3]) > rating:
                rating = int(app[3])
                app_2 = app
    if app_2 not in apps_non_duplicates:
        apps_non_duplicates.append(app_2)
    
for name in unique_apps:
    for app in apps_data_google:
        if name == app[0]:
            apps_non_duplicates.append(app)

Agora perceba que os apps não estão duplicados:

In [0]:
for app in apps_non_duplicates:
    name = app[0]
    if name == "Instagram":
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


##Removendo aplicativos que não estão em inglês

Agora perceba que alguns apps do Google Play não estão em inglês:

In [0]:
for app in apps_data_apple:
    name = app[1]
    if name == "搜狐视频HD-欢乐颂2 全网首播":
        print(app)

['414430589', '搜狐视频HD-欢乐颂2 全网首播', '79696896', 'USD', '0.0', '3768', '5', '4.0', '3.0', '6.6', '17+', 'Entertainment', '24', '5', '1', '1']


E alguns da Apple store também:

In [0]:
for app in apps_non_duplicates:
    name = app[0]
    if name == 'Flame - درب عقلك يوميا':
        print(app)

['Flame - درب عقلك يوميا', 'EDUCATION', '4.6', '56065', '37M', '1,000,000+', 'Free', '0', 'Everyone', 'Education', 'July 26, 2018', '3.3', '4.1 and up']


Para remover esses aplicativos analizamos se existem mais de três caracteres no nome que não pertencem a tabela ASCII, ou seja, se esses caracteres corresponderem a um valor maior do que 127.

In [0]:
apps_english_apple = []
for app in apps_data_apple:
    name = app[1]
    i = 0
    for caracter in name:
        if ord(caracter) > 127:
            i += 1
    if i < 3:
        apps_english_apple.append(app)

In [0]:
apps_english_google = []
for app in apps_non_duplicates:
    name = app[0]
    i = 0
    for caracter in name:
        if ord(caracter) > 127:
            i += 1
    if i < 3:
        apps_english_google.append(app)

Agora perceba que os apps que não tem nome em inglês foram removidos da Apple Store

In [0]:
non_english_app = False
for app in apps_english_apple:
    name = app[1]
    if name == "搜狐视频HD-欢乐颂2 全网首播":
        non_english_app = True
if (not non_english_app):
    print("Os apps que não tem nome em inglês foram removidos da Apple Store")

Os apps que não tem nome em inglês foram removidos da Apple Store


Agora perceba que os apps que não tem nome em inglês foram removidos da Google Play

In [0]:
non_english_app = False
for app in apps_english_google:
    name = app[0]
    if name == 'Flame - درب عقلك يوميا':
        non_english_app = True
if (not non_english_app):
    print("Os apps que não tem nome em inglês foram removidos do Google Play")

Os apps que não tem nome em inglês foram removidos do Google Play


## Isolando os aplicativos gratuitos

Nas células abaixo, isolamos os aplicativos, tanto da Apple Store quanto do Google Play, que são gratuitos e os armazenamos em uma lista para cada conjunto de dados.

In [0]:
free_apps_apple = []
for app in apps_english_apple:
    price = app [4]
    if price == '0.0':
        free_apps_apple.append(app) 

In [0]:
free_apps_google = []
for app in apps_english_google:
    price = app [7]
    if price == '0':
        free_apps_google.append(app)

Com a finalidade de sabermos quantos aplicativos ainda possuímos em cada conjunto de dados, usamos a função len().


In [0]:
len(free_apps_apple)

3203

In [0]:
len(free_apps_google)

8845

## Aplicativos mais comuns por gênero

Nosso objetivo é determinar os tipos de aplicativos que são mais prováveis de atrair usuários, pois nossa receita é altamente influenciada pela quantidade de pessoas que usam nossos aplicativos.

Para minimizar riscos e sobrecarga, a nossa estratégia de validação para uma ideia de aplicativo é compreendida em três passos:

- Construir uma versão mínima em Android do aplicativo e o adicionamos ao Google Play;
- Se o aplicativo tem uma boa resposta dos usuários, então nós iremos desenvolvê-lo mais;
- Se o aplicativo é útil depois de seis meses, nós também construímos uma versão do aplicativo para iOS e o adicionamos ao App Store.

Note que criamos uma tabela de frequência com um dicionário para armazenar a coluna "**prime_genre**'', do conjunto de dados do Apple Store.

In [0]:
genres_apple_dict = {}
for app in free_apps_apple:
    genre_apple = app[11]
    if genre_apple in genres_apple_dict:
        genres_apple_dict[genre_apple] += 1
    else:
        # create and assign value 1
        genres_apple_dict[genre_apple] = 1

In [0]:
genres_apple_dict

{'Book': 12,
 'Business': 17,
 'Catalogs': 4,
 'Education': 118,
 'Entertainment': 251,
 'Finance': 35,
 'Food & Drink': 26,
 'Games': 1866,
 'Health & Fitness': 65,
 'Lifestyle': 50,
 'Medical': 6,
 'Music': 66,
 'Navigation': 6,
 'News': 43,
 'Photo & Video': 160,
 'Productivity': 56,
 'Reference': 17,
 'Shopping': 83,
 'Social Networking': 106,
 'Sports': 69,
 'Travel': 40,
 'Utilities': 79,
 'Weather': 28}

Em seguida, é calculado e mostrado a porcentagem dos aplicativos de acordo com o número total de aplicativos da Apple Store depois de retirarmos os que não são ingleses e não gratuitos

In [0]:
genres_apple_dict
total_number_of_apps_apple = 3203
genres_proportions_apple = {}

for key in genres_apple_dict:
    proportion_apple = genres_apple_dict[key] / total_number_of_apps_apple
    genres_proportions_apple[key] = proportion_apple

print (genres_proportions_apple)
print ("\n")
print (genres_apple_dict)

{'Social Networking': 0.033093974399000935, 'Photo & Video': 0.049953168904152356, 'Games': 0.5825788323446769, 'Music': 0.020605682172962846, 'Reference': 0.005307524196066188, 'Health & Fitness': 0.020293474867311895, 'Weather': 0.008741804558226662, 'Utilities': 0.024664377146425227, 'Travel': 0.012488292226038089, 'Shopping': 0.025913206369029034, 'News': 0.013424914142990947, 'Navigation': 0.0018732438339057135, 'Lifestyle': 0.015610365282547611, 'Entertainment': 0.07836403371838901, 'Food & Drink': 0.008117389946924758, 'Sports': 0.021542304089915705, 'Book': 0.003746487667811427, 'Finance': 0.010927255697783328, 'Education': 0.036840462066812366, 'Productivity': 0.017483609116453323, 'Business': 0.005307524196066188, 'Catalogs': 0.001248829222603809, 'Medical': 0.0018732438339057135}


{'Social Networking': 106, 'Photo & Video': 160, 'Games': 1866, 'Music': 66, 'Reference': 17, 'Health & Fitness': 65, 'Weather': 28, 'Utilities': 79, 'Travel': 40, 'Shopping': 83, 'News': 43, 'Nav

Agora, criamos uma tabela de frequência com um dicionário para armazenar a coluna "**Category**'', do conjunto de dados do Google Play.

In [0]:
genres_google_dict = {}
for app in free_apps_google:
    genre_google = app[1]
    if genre_google in genres_google_dict:
        genres_google_dict[genre_google] += 1
    else:
        genres_google_dict[genre_google] = 1

In [0]:
genres_google_dict

{'ART_AND_DESIGN': 57,
 'AUTO_AND_VEHICLES': 82,
 'BEAUTY': 53,
 'BOOKS_AND_REFERENCE': 189,
 'BUSINESS': 407,
 'COMICS': 54,
 'COMMUNICATION': 286,
 'DATING': 165,
 'EDUCATION': 103,
 'ENTERTAINMENT': 85,
 'EVENTS': 63,
 'FAMILY': 1676,
 'FINANCE': 328,
 'FOOD_AND_DRINK': 110,
 'GAME': 858,
 'HEALTH_AND_FITNESS': 273,
 'HOUSE_AND_HOME': 71,
 'LIBRARIES_AND_DEMO': 83,
 'LIFESTYLE': 344,
 'MAPS_AND_NAVIGATION': 123,
 'MEDICAL': 310,
 'NEWS_AND_MAGAZINES': 248,
 'PARENTING': 58,
 'PERSONALIZATION': 294,
 'PHOTOGRAPHY': 261,
 'PRODUCTIVITY': 345,
 'SHOPPING': 199,
 'SOCIAL': 236,
 'SPORTS': 300,
 'TOOLS': 748,
 'TRAVEL_AND_LOCAL': 207,
 'VIDEO_PLAYERS': 159,
 'WEATHER': 70}

Abaixo é calculado e mostrado a porcentagem dos aplicativos da coluna **Category** de acordo com o número total de aplicativos do Google Play depois de retirarmos os que não são ingleses e não gratuitos.

In [0]:
genres_google_dict
total_number_of_apps_google = 8845
genres_proportions_google = {}

for key in genres_google_dict:
  proportion_google = genres_google_dict[key] / total_number_of_apps_google
  genres_proportions_google[key] = proportion_google

print (genres_proportions_google)
print ("\n")
print (genres_google_dict)

{'BUSINESS': 0.04601469756924816, 'COMMUNICATION': 0.03233465234595817, 'DATING': 0.018654607122668174, 'EDUCATION': 0.011644997173544376, 'FAMILY': 0.1894855850763143, 'ENTERTAINMENT': 0.009609949123798756, 'FINANCE': 0.03708309779536461, 'FOOD_AND_DRINK': 0.01243640474844545, 'HEALTH_AND_FITNESS': 0.03086489542114189, 'HOUSE_AND_HOME': 0.008027133973996608, 'LIFESTYLE': 0.03889202939513849, 'GAME': 0.09700395703787451, 'MEDICAL': 0.035048049745618995, 'SOCIAL': 0.026681741096664782, 'SHOPPING': 0.022498586772187676, 'PHOTOGRAPHY': 0.029508196721311476, 'SPORTS': 0.033917467495760314, 'TRAVEL_AND_LOCAL': 0.023403052572074617, 'TOOLS': 0.08456755228942905, 'PERSONALIZATION': 0.03323911814584511, 'PRODUCTIVITY': 0.03900508762012436, 'VIDEO_PLAYERS': 0.017976257772752968, 'NEWS_AND_MAGAZINES': 0.028038439796495195, 'BOOKS_AND_REFERENCE': 0.021368004522329, 'ART_AND_DESIGN': 0.00644431882419446, 'MAPS_AND_NAVIGATION': 0.01390616167326173, 'COMICS': 0.006105144149236857, 'WEATHER': 0.00791

No código abaixo, criamos um segundo dicionário para armazenar os gêneros da coluna ''**Genres**'', também do Google Play.

In [0]:
genres_google_dict2 = {}
for app in free_apps_google:
    genre_google = app[9]
    if genre_google in genres_google_dict:
        genres_google_dict2[genre_google] += 1
    else:
        genres_google_dict2[genre_google] = 1

In [0]:
genres_google_dict2

{'Action': 1,
 'Action;Action & Adventure': 1,
 'Adventure': 1,
 'Adventure;Action & Adventure': 1,
 'Adventure;Education': 1,
 'Arcade': 1,
 'Arcade;Action & Adventure': 1,
 'Arcade;Pretend Play': 1,
 'Art & Design': 1,
 'Art & Design;Action & Adventure': 1,
 'Art & Design;Creativity': 1,
 'Art & Design;Pretend Play': 1,
 'Auto & Vehicles': 1,
 'Beauty': 1,
 'Board': 1,
 'Board;Action & Adventure': 1,
 'Board;Brain Games': 1,
 'Books & Reference': 1,
 'Books & Reference;Education': 1,
 'Business': 1,
 'Card': 1,
 'Card;Action & Adventure': 1,
 'Casino': 1,
 'Casual': 1,
 'Casual;Action & Adventure': 1,
 'Casual;Brain Games': 1,
 'Casual;Creativity': 1,
 'Casual;Education': 1,
 'Casual;Music & Video': 1,
 'Casual;Pretend Play': 1,
 'Comics': 1,
 'Comics;Creativity': 1,
 'Communication': 1,
 'Communication;Creativity': 1,
 'Dating': 1,
 'Education': 1,
 'Education;Action & Adventure': 1,
 'Education;Brain Games': 1,
 'Education;Creativity': 1,
 'Education;Education': 1,
 'Education;Musi

Abaixo é calculado e mostrado a porcentagem dos aplicativos da coluna **Genres** de acordo com o número total de aplicativos do Google Play depois de retirarmos os que não são ingleses e não gratuitos

In [0]:
genres_google_dict2
total_number_of_apps_google = 8845
genres_proportions_google2 = {}

for key in genres_google_dict2:
  proportion_google2 = genres_google_dict2[key] / total_number_of_apps_google
  genres_proportions_google2[key] = proportion_google2

print (genres_proportions_google2)
print ("\n")
print (genres_google_dict2)

{'Business': 0.00011305822498586772, 'Communication': 0.00011305822498586772, 'Dating': 0.00011305822498586772, 'Education': 0.00011305822498586772, 'Education;Education': 0.00011305822498586772, 'Entertainment': 0.00011305822498586772, 'Entertainment;Music & Video': 0.00011305822498586772, 'Finance': 0.00011305822498586772, 'Food & Drink': 0.00011305822498586772, 'Health & Fitness': 0.00011305822498586772, 'House & Home': 0.00011305822498586772, 'Lifestyle': 0.00011305822498586772, 'Arcade': 0.00011305822498586772, 'Adventure;Action & Adventure': 0.00011305822498586772, 'Casual': 0.00011305822498586772, 'Sports': 0.00011305822498586772, 'Strategy': 0.00011305822498586772, 'Word': 0.00011305822498586772, 'Puzzle': 0.00011305822498586772, 'Action': 0.00011305822498586772, 'Music': 0.00011305822498586772, 'Simulation': 0.00011305822498586772, 'Adventure': 0.00011305822498586772, 'Role Playing': 0.00011305822498586772, 'Racing': 0.00011305822498586772, 'Card': 0.00011305822498586772, 'Cas

Agora, criamos uma função para gerar uma tabela de frequência e, em seguida, usá-la em conjunto com a função **display_table()**.

In [0]:
def freq_table (dataset, index):
    genres_dataset_dict = {}  
    for row in dataset:
        dataset_genre = row[index]
        if dataset_genre in genres_dataset_dict:
            genres_dataset_dict[dataset_genre] += 1
        else:
            genres_dataset_dict[dataset_genre] = 1 
    return genres_dataset_dict  

In [0]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

Agora, analisamos os gêneros mais comuns dos aplicativos da Apple Store:

In [0]:
display_table (free_apps_apple, 11)

Games : 1866
Entertainment : 251
Photo & Video : 160
Education : 118
Social Networking : 106
Shopping : 83
Utilities : 79
Sports : 69
Music : 66
Health & Fitness : 65
Productivity : 56
Lifestyle : 50
News : 43
Travel : 40
Finance : 35
Weather : 28
Food & Drink : 26
Reference : 17
Business : 17
Book : 12
Navigation : 6
Medical : 6
Catalogs : 4



Analisando a tabela de frequência gerada para a coluna **prime_genre** do conjunto de dados da Apple Store, concluímos que  o gênero mais comum é o de **Games** e o segundo mais comum é o de **Entertainment**, com isso, é perceptível que o padrão de busca dos usuários é focado em aplicativos voltados para o lazer, a diversão, o descanso, mas também para o conhecimento (educação) e, por isso, têm-se mais aplicativos projetados para a diversão do usuário. Além disso, não é possível recomendar um perfil de aplicativo para o mercado da Apple Store se baseando apenas por esta tabela de frequência, devido ao fato de que nem todos os aplicativos de determinado gênero têm um grande número de usuário, pois ele pode não agradar os usuários.


Agora, os gêneros dos aplicativos do Google Play que se encontram na coluna ''**Category**'':

In [0]:
display_table (free_apps_google, 1)

FAMILY : 1676
GAME : 858
TOOLS : 748
BUSINESS : 407
PRODUCTIVITY : 345
LIFESTYLE : 344
FINANCE : 328
MEDICAL : 310
SPORTS : 300
PERSONALIZATION : 294
COMMUNICATION : 286
HEALTH_AND_FITNESS : 273
PHOTOGRAPHY : 261
NEWS_AND_MAGAZINES : 248
SOCIAL : 236
TRAVEL_AND_LOCAL : 207
SHOPPING : 199
BOOKS_AND_REFERENCE : 189
DATING : 165
VIDEO_PLAYERS : 159
MAPS_AND_NAVIGATION : 123
FOOD_AND_DRINK : 110
EDUCATION : 103
ENTERTAINMENT : 85
LIBRARIES_AND_DEMO : 83
AUTO_AND_VEHICLES : 82
HOUSE_AND_HOME : 71
WEATHER : 70
EVENTS : 63
PARENTING : 58
ART_AND_DESIGN : 57
COMICS : 54
BEAUTY : 53


Também do Google Play, mas os que se encontram na coluna ''**Genres**":

In [0]:
display_table (free_apps_google, 9)

Tools : 747
Entertainment : 538
Education : 474
Business : 407
Productivity : 345
Lifestyle : 343
Finance : 328
Medical : 310
Sports : 306
Personalization : 294
Communication : 286
Action : 274
Health & Fitness : 273
Photography : 261
News & Magazines : 248
Social : 236
Travel & Local : 206
Shopping : 199
Books & Reference : 189
Simulation : 181
Dating : 165
Arcade : 163
Video Players & Editors : 157
Casual : 156
Maps & Navigation : 123
Food & Drink : 110
Puzzle : 100
Racing : 88
Role Playing : 83
Libraries & Demo : 83
Auto & Vehicles : 82
Strategy : 81
House & Home : 71
Weather : 70
Events : 63
Adventure : 59
Comics : 53
Beauty : 53
Art & Design : 53
Parenting : 44
Card : 40
Trivia : 37
Casino : 37
Educational;Education : 35
Board : 34
Educational : 33
Education;Education : 30
Word : 23
Casual;Pretend Play : 21
Music : 18
Racing;Action & Adventure : 15
Puzzle;Brain Games : 15
Entertainment;Music & Video : 15
Casual;Brain Games : 12
Casual;Action & Adventure : 12
Arcade;Action & Advent

Ao analisar as tabelas de frequência geradas para as colunas **Genre** e **Category** do conjunto de dados do Google Play, concluímos que os gêneros mais comuns são, respectivamente, **Tools** e **Family**. Ademais, é notório que o padrão de busca dos usuários é bem variado, pois é dividida tanto para a diversão quanto para a fins práticos, diferentemente dos usuários da Apple Store, que buscam por mais  aplicativos divertidos. Baseando-se no que vimos até agora, não é possível recomendar um perfil de aplicativo, pois nem todos os aplicativos de determinado gênero atraem os usuários. Pelas tabelas de frequência geradas, é possível saber quais são os gêneros de aplicativos mais comuns e quais deles possuem mais usuários de cada coluna, porém, para ter uma visão geral, deve-se analisar as duas colunas em conjunto.

##Aplicativos mais populares por gênero da Apple Store

Nos códigos abaixo, iremos calcular o número médio de classificações de usuários por gênero de aplicativo da Apple Store, para isso, criamos um dicionário, nas quais temos as chaves representando os gêneros e os números representando as médias.

In [0]:
genres_apple_dict = {}
for app in free_apps_apple:
    genre_apple = app[11]
    if genre_apple in genres_apple_dict:
        genres_apple_dict[genre_apple] += 1
    else:
        genres_apple_dict[genre_apple] = 1

In [0]:
rating_count_tot_dict =  {}
for app in free_apps_apple:
  rating_count_tot = app[11]
  if rating_count_tot in rating_count_tot_dict:
    rating_count_tot_dict[rating_count_tot] += int(app[5])
  else:
    rating_count_tot_dict[rating_count_tot] = int(app[5])
for app in rating_count_tot_dict:
  rating_count_tot_dict[app] = rating_count_tot_dict[app] / genres_apple_dict[app]

Em seguida, transformamos o dicionário criado acima em uma lista de tuplas e os reorganizamos pelo valor da média:

In [0]:
rating_count_tot_tuples = []
for app in rating_count_tot_dict:
  tuples = (rating_count_tot_dict[app], app)
  rating_count_tot_tuples.append(tuples)

In [0]:
rating_count_tot_tuples = sorted(rating_count_tot_tuples, reverse = True)
for app in rating_count_tot_tuples:
  print(app)

(86090.33333333333, 'Navigation')
(79350.4705882353, 'Reference')
(71548.34905660378, 'Social Networking')
(57326.530303030304, 'Music')
(52279.892857142855, 'Weather')
(46384.916666666664, 'Book')
(33333.92307692308, 'Food & Drink')
(32367.02857142857, 'Finance')
(28441.54375, 'Photo & Video')
(28243.8, 'Travel')
(27230.734939759037, 'Shopping')
(23298.015384615384, 'Health & Fitness')
(23008.898550724636, 'Sports')
(22886.36709539121, 'Games')
(21248.023255813954, 'News')
(21028.410714285714, 'Productivity')
(19156.493670886077, 'Utilities')
(16815.48, 'Lifestyle')
(14195.358565737051, 'Entertainment')
(7491.117647058823, 'Business')
(7003.983050847458, 'Education')
(4004.0, 'Catalogs')
(612.0, 'Medical')


Ao analisar os resultados, chegamos à conclusão de que um perfil de aplicativo indicado para a Apple Store seria os de navegação, que ajudam o usuário a melhor transitar pela cidade ou em outros locais, além de ajudá-lo a não se perder em um local desconhecido.

## Aplicativos mais populares por gênero do Google Play

Nos códigos abaixo, iremos calcular o número médio de classificações de usuários por gênero de aplicativo do Google Play, para isso, novamente, criamos um dicionário, nas quais temos as chaves representando os gêneros e os números representando as médias.

In [0]:
genres_google_dict = {}
for app in free_apps_google:
    genre_google = app[1]
    if genre_google in genres_google_dict:
        genres_google_dict[genre_google] += 1
    else:
        genres_google_dict[genre_google] = 1

In [0]:
Installs_dict =  {}
for app in free_apps_google:
  Installs = app[1]
  app[5] = app[5].replace('+', '')
  app[5] = app[5].replace(',', '')
  if Installs in Installs_dict:
    Installs_dict[Installs] += int(app[5])
  else:
    Installs_dict[Installs] = int(app[5])
for app in Installs_dict:
  Installs_dict[app] = Installs_dict[app] / genres_google_dict[app]

Assim como fizemos no tópico anterior, para os aplicativos da Apple Store, transformamos o dicionário acima em uma lista de tuplas e os reorganizamos pelo valor da média:

In [0]:
Installs_tuples = []
for app in rating_count_tot_dict:
  tuples = (rating_count_tot_dict[app], app)
  Installs_tuples.append(tuples)

In [0]:
Installs_tuples = sorted(Installs_tuples, reverse = True)
for app in Installs_tuples:
  print(app)

(86090.33333333333, 'Navigation')
(79350.4705882353, 'Reference')
(71548.34905660378, 'Social Networking')
(57326.530303030304, 'Music')
(52279.892857142855, 'Weather')
(46384.916666666664, 'Book')
(33333.92307692308, 'Food & Drink')
(32367.02857142857, 'Finance')
(28441.54375, 'Photo & Video')
(28243.8, 'Travel')
(27230.734939759037, 'Shopping')
(23298.015384615384, 'Health & Fitness')
(23008.898550724636, 'Sports')
(22886.36709539121, 'Games')
(21248.023255813954, 'News')
(21028.410714285714, 'Productivity')
(19156.493670886077, 'Utilities')
(16815.48, 'Lifestyle')
(14195.358565737051, 'Entertainment')
(7491.117647058823, 'Business')
(7003.983050847458, 'Education')
(4004.0, 'Catalogs')
(612.0, 'Medical')


Ao analisar os resultados, chegamos à conclusão de que um perfil de aplicativo indicado para Google Play também seria os de navegação, que ajudam o usuário a melhor transitar pela cidade ou em outros locais, além de ajudá-lo a não se perder em um local desconhecido.